# 20211230 - split in groups + per group editing

## split into groups

In [24]:
import json

s = [json.loads(i) for i in open('sentences.json')]
!rm -Rf chunks; mkdir -p chunks

# chunk in pieces of N = 30 sentences
N = 30
c = 0
while s:
    c = c + 1
    chunk = s[:N]
    with open(f'chunks/{c}.txt', 'w') as f:
        for i in chunk:
            f.write(f"E|{i['en']}\nP|{i['pl']}\n\n\n")      
    s = s[N:]

## hand-fix groups
here I can fix groups by hand and overwrite the txt

In [42]:
import random

# read file, do a re-match, write file again
# if lines are not E| or P|, -> rematch them
for i in [3]:
    fn = f'chunks/{i}.txt'
    lines = open(fn).readlines()

    # grab unmatched lines
    unmatched = {
        'pl': [],
        'en': []
    }
    new_lines = []
    for line in lines:
        if line[:2] == 'E|':
            en = line[2:].strip()
            curr = 'en'
        elif line[:2] == 'P|':
            # the pairs come EN -> PL, therefore write in PL
            curr = 'pl'
            new_lines.append({'en':en, 'pl':line[2:].strip()})
        elif line.strip():
            unmatched[curr].append(line)

    assert len(unmatched['pl']) == len(unmatched['en']), f'error with file {fn}'
    new_lines.extend(
        {'en':en.strip(), 'pl':pl.strip()}
        for en, pl in
        zip(unmatched['en'], unmatched['pl'])
    )

    random.shuffle(new_lines)
    with open(fn, 'w') as f:
        for i in new_lines:
            f.write(f"E|{i['en']}\nP|{i['pl']}\n\n\n") 
    break

        

    

# 20211220 - Extract sentences
## paragraphs

In [1]:
is_pl = True
par = {}
pars = []
for line in open('text'):
    k = 'pl' if is_pl else 'en'
    par[k] = par.get(k, [])
    if line == '\n':
        par[k] = " ".join(par[k])
        
        if not is_pl:
            # the pars go: pl, en, pl, en
            pars.append(par)
            par = {}
            
        is_pl = not is_pl
        continue
    par[k].append(line.strip())
    
import json

with open('pars', 'w') as f:
    for par in pars:
        f.write(json.dumps(par).replace('…','...'))
        f.write('\n')

## pair sentences

In [7]:

def fix2(x):
    return [i for i in x if i]

a = 0
with open('sen', 'w') as f:
    b = 0
    for par in open('pars'):
        b = b + 1
        par = json.loads(par)
        en = fix2(par['en'].split('.'))
        pl = fix2(par['pl'].split('.'))
        
        if len(pl) != len(en):
            a = a + 1
            print('------')
            [print(i) for i in pl]
            print('---')
            [print(i) for i in en]
            assert 0
        
        for e, p in zip(en, pl):
            d = {
                'en': e.strip(),
                'pl': p.strip()
            }
            f.write(json.dumps(d))
            f.write('\n')


In [6]:
open('sen').read()

'{"en": "It was a cold, foggy day", "pl": "By\\u0142 mglisty, zimny poranek"}\n{"en": " Looking at the street full of people rushing to their work, Tomek imagined his dream job", "pl": " Patrz\\u0105c na ulic\\u0119 pe\\u0142n\\u0105 ludzi spiesz\\u0105cych si\\u0119 do pracy, Tomek wyobrazi\\u0142 sobie swoj\\u0105 wymarzon\\u0105 posad\\u0119"}\n{"en": " He was sitting at his desk and drinking freshly ground coffee", "pl": " Siedzia\\u0142 przy biurku na najwy\\u017cszym pi\\u0119trze wie\\u017cowca i popija\\u0142 \\u015bwie\\u017c\\u0105 kaw\\u0119"}\n{"en": "Suddenly, the traffic lights turned green", "pl": "Wtem, \\u015bwiat\\u0142o na ulicy zmieni\\u0142o si\\u0119 na zielone"}\n{"en": " Tomek was really close to the place which he had imagined working in", "pl": " Tomek by\\u0142 ju\\u017c o krok od miejsca, w kt\\u00f3rym wyobra\\u017ca\\u0142 sobie swoj\\u0105 przysz\\u0142\\u0105 prac\\u0119"}\n{"en": " Before all that, however, he had to pass the job interview, which was to

In [274]:
en

['Agnes couldn’t believe it',
 ' It was like a dream come true',
 ' Since that day, her idol was her best friend',
 ' Agnes was at every concert of his and spent time with him',
 ' She always had a place backstage']